In [3]:
pip install pydantic controlflow fastapi python-dotenv

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [1]:
from dotenv import load_dotenv
load_dotenv("creds.env")

False

In [20]:
import controlflow as cf

from agents import (create_agent,leader,council_member1,
                    council_member2,council_member3,coder,
                    agent_maker)

from models import AgentParams


#only one agent per turn. Keep responses to 1 sentence max.
@cf.flow
def council_task(task: str):

    #step 1: deliberation and voting process
    deliberate = cf.run(
        "Deliberate on best way to complete task and hold a vote.",
        agents=[leader, council_member1,council_member2,council_member3],
        completion_agents=[leader],
        instructions="""
            Deliberate with other council members on best way to complete task.
            Allow each council member to provide input before voting.
            Vote on best answer.
            Show each council members vote and recommendation.
            Show the entire deliberation and voting process.
            Show the final decision and the reasoning behind it.
            """,
        context={"task": task},
        result_type=str
    )
    print(deliberate)
    
    # Step 2: write code for task
    codes = cf.run(
        "write code for task",
        agents=[coder],
        instructions="""
            task coder agent to provide code for task.
            convert the code into a readable string
            provide code in a format that can be parsed by python with ast.literal_eval().
            """,
        context={"deliberation":deliberate},
        result_type=str
    )
    #print(codes)

    # Step 3: generate an agent to run the code
    custom_agent_params = cf.run(
        "Create a ControlFlow agent using the provided code.",
        agents=[agent_maker],
        context={"code": codes},
        result_type=AgentParams
    )
    custom_agent = create_agent(custom_agent_params)
    
    # step 4: run the agent
    result = cf.run(
    "complete the task using the provided code.",
    agents=[custom_agent],
    #tools=[code.python],

    )

    return result